In [ ]:
import boto3
import os

In [ ]:
# Connect to AWS
# Make sure you have the AWS credentials in your .secret file
with open("/home/dbogdoll/mcity_data_engine/.secret", "r") as file:
    for line in file:
        key, value = line.strip().split("=")
        os.environ[key] = value

s3 = boto3.client(
    "s3",
    aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID", None),
    aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY", None),
)

In [ ]:
def _process_aws_result(result):
    # Get list of folders from AWS response
    if "CommonPrefixes" in result:
        folders = [prefix["Prefix"] for prefix in result["CommonPrefixes"]]
        return folders
    else:
        return None

In [ ]:
def list_files(bucket, prefix):
    files = []
    paginator = s3.get_paginator('list_objects_v2')
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        if 'Contents' in page:
            for obj in page['Contents']:
                files.append(obj['Key'])
    
    return files

In [ ]:
bucket = "mcity-data-engine"
prefix = ""
all_files = list_files(bucket, prefix)

print(f"Found {len(all_files)} files in {bucket}/{prefix}")

download_path = "/media/dbogdoll/Datasets/data_engine_rolling_processed"
for file in all_files:
    file_path = os.path.join(download_path, file)
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    s3.download_file(bucket, file, file_path)
    print(f"Downloaded {file} to {file_path}")